In [1]:
import json
import urllib
import glob
import os
import time
import xml.etree.ElementTree as ET

In [2]:
user="aa.annaaniol@gmail.com" 
lpmn="any2txt|wcrft2" 
url="http://ws.clarin-pl.eu/nlprest2/base" 

In [70]:
potop_file = "../lab2/sources/potop.txt"
named_entity_label = 'ign'

In [4]:
with open (potop_file, "rb") as myfile:
    doc = myfile.read()
    upload_request = urllib.request.Request(url + '/upload/', doc, {'Content-Type': 'binary/octet-stream'})
    fileid = urllib.request.urlopen(upload_request).read().decode("utf-8") 

In [5]:
print(fileid)

/users/default/9d01b097-b264-4018-95d7-f4ae6a24c016


In [6]:
data = {'lpmn': lpmn, 'user': user, 'file':fileid}
# doc = urllib.parse.urlencode(data)
# binary_doc = doc.encode('utf-8')

doc = json.dumps(data)
binary_doc = doc.encode("utf-8")
print(binary_doc)


# data = urllib.parse.urlencode(data).encode("utf-8")
# print(data)

b'{"lpmn": "any2txt|wcrft2", "user": "aa.annaaniol@gmail.com", "file": "/users/default/9d01b097-b264-4018-95d7-f4ae6a24c016"}'


In [7]:
task_request = urllib.request.Request(url + '/startTask/', binary_doc, {'Content-Type': 'application/json'})

In [8]:
taskid = urllib.request.urlopen(task_request).read()
taskid = taskid.decode("utf-8")

In [9]:
status_request = urllib.request.Request(url + '/getStatus/' + taskid)

In [11]:
# wait a moment for processing to be done and run

response = urllib.request.urlopen(status_request);
data = json.load(response)
print(data)

{'value': [{'name': 'file', 'fileID': '/requests/wcrft2/fad4a5f8-cdd8-43ac-b625-0628a43d53b0'}], 'status': 'DONE'}


In [12]:
file_id = data['value'][0]['fileID']
print(file_id)

/requests/wcrft2/fad4a5f8-cdd8-43ac-b625-0628a43d53b0


In [13]:
content_request = urllib.request.Request(url + '/download' + file_id)

In [14]:
content = urllib.request.urlopen(content_request).read()
content = content.decode("utf-8")

In [68]:
print(content[:1000])

<?xml version="1.0" encoding="UTF-8"?>
<!DOCTYPE chunkList SYSTEM "ccl.dtd">
<chunkList>
 <chunk id="ch1" type="p">
  <sentence id="s1">
   <tok>
    <orth>frame</orth>
    <lex disamb="1"><base>frame</base><ctag>ign</ctag></lex>
   </tok>
  </sentence>
 </chunk>
 <chunk id="ch2" type="p">
  <sentence id="s2">
   <tok>
    <orth>Henryk</orth>
    <lex disamb="1"><base>Henryk</base><ctag>subst:sg:nom:m1</ctag></lex>
   </tok>
   <tok>
    <orth>Sienkiewicz</orth>
    <lex disamb="1"><base>Sienkiewicz</base><ctag>subst:sg:nom:m1</ctag></lex>
   </tok>
   <tok>
    <orth>Potop</orth>
    <lex disamb="1"><base>potop</base><ctag>subst:sg:nom:m3</ctag></lex>
   </tok>
  </sentence>
 </chunk>
 <chunk id="ch3" type="p">
  <sentence id="s3">
   <tok>
    <orth>tom</orth>
    <lex disamb="1"><base>tom</base><ctag>subst:sg:nom:m3</ctag></lex>
   </tok>
   <tok>
    <orth>I</orth>
    <lex disamb="1"><base>i</base><ctag>conj</ctag></lex>
   </tok>
  </sentence>
 </chunk>
 <chunk id="ch4" type="p">

In [16]:
root = ET.fromstring(content)

In [17]:
root.tag

'chunkList'

In [18]:
root.attrib

{}

In [65]:
named_entities = []

In [73]:
for chunk in root:
    for sentence in chunk:
        for tok in sentence:
            if_named_entity = False
            for lex in tok:
                if lex != None:
                    ctag = lex.find('ctag')
                    if ctag != None:
                        roles = ctag.text.split(':')
                        if named_entity_label in roles:
                            if_named_entity = True
                    base = lex.find('base')
                    if base != None and if_named_entity:
#                         print("named entity:", base.text)
                        named_entities.append(base.text)

                        

In [74]:
len(named_entities)

27429